If you are not using GitHub [click here to open the notebook using nbviewer](http://nbviewer.jupyter.org/github/knightlab-analyses/intro-stats-2016/blob/master/week3/week3-distances-and-ordinations.ipynb).

In [1]:
%matplotlib inline
from __future__ import division

from emperor import Emperor

from biom.util import biom_open
from biom import load_table

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from skbio import TreeNode
from skbio.stats.ordination import pcoa
from skbio.diversity import beta_diversity
from skbio.stats.distance import anosim, permanova
from scipy.spatial.distance import euclidean, braycurtis, jaccard, canberra

import qiime_default_reference

plt.style.use('ggplot')

Let's define a dictionary of the metrics we want to use, note that we define unweighted and weighted UniFrac as string to let scikit-bio use the optimized versions.

In [2]:
metrics = {'Unweighted UniFrac': 'unweighted_unifrac',
           'Weighted UniFrac': 'weighted_unifrac',
           'Jaccard': jaccard,
           'Canberra': canberra,
           'Bray-Curtis': braycurtis}

# Loading your input data

Before we can get started, we need to load three files:

- OTU table.
- Sample information.
- Phylogenetic tree 🌴 (only relevant for the phylogenetic metrics).

For this example, we will use the data from [Fierer et al. 2010](http://www.pnas.org/content/107/14/6477.full) (the data was retrieved from study [232](https://qiita.ucsd.edu/study/description/232) in [Qiita](https://qiita.ucsd.edu), remember you need to be logged in to access the study).

-----------

Load your mapping file with pandas.

In [3]:
mf = pd.read_csv('data/mapping-file.txt', dtype=str, keep_default_na=False,
                 na_values=[], sep='\t')
mf.set_index('#SampleID', inplace=True)

Read the Greengenes tree (may take a few seconds to load).

In [4]:
tree = TreeNode.read(qiime_default_reference.get_reference_tree())

for n in tree.traverse():
    if n.length is None:
        n.length = 0

Load the BIOM table and create a matrix of the counts, rarefied at 15,000 sequences per sample.

In [5]:
bt = load_table('data/otu-table.biom')
bt.subsample(15000)

data = np.array([bt.data(i) for i in bt.ids()], dtype='int64')

scikit-bio provides a high-level function, `beta_diversity`, to compute the distance matrix of a table of counts given a distance metric.

Here, we calculate all the distance matrices (in each loop we calculate a metric), and then compute the principal coordinates analysis with the `pcoa` function.

In [6]:
ordinations = {}

for m, f in metrics.items():
    if m in {'Unweighted UniFrac', 'Weighted UniFrac'}:
        x = beta_diversity(f, data, bt.ids(), otu_ids=bt.ids('observation'), tree=tree)
    else:
        x = beta_diversity(f, data, bt.ids())

    ordinations[m] = pcoa(x)

/Users/yoshikivazquezbaeza/miniconda/envs/3mperor/lib/python3.5/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:102: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.050376995226319214 and the largest is 11.704923569938886.
  RuntimeWarning
/Users/yoshikivazquezbaeza/miniconda/envs/3mperor/lib/python3.5/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:102: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. S

Let's visualize one of these ordinations.

In [7]:
Emperor(ordinations['Unweighted UniFrac'], mf, remote=True)